In [ ]:
import pandas as pd

In [ ]:
df = pd.read_pickle('datasets/yelp/business.pkl')

Por alguna Razón cargo 28 Columnas.. Las 14 Con datos reales y 14 con datos no reales

In [ ]:
business = df.iloc[:,0:14]

del df  # Borro esta variable para liberar memoria

Verificando los NaN en el Dataframe

In [ ]:
business.isna().sum()

,0
business_id,0
name,0
address,0
city,0
state,3
postal_code,0
latitude,0
longitude,0
stars,0
review_count,0


Estos son los rellenos que voy a utilizar en las columnas que tienen NaN a fin de mantener las estructuras y luego una lógica de desempaquetado pueda ser aplicada

Relleno Todos los NaN del Dataframe con 'No Disponible' para evitar inconvenientes futuros

In [ ]:
business.loc[:, 'state'] = business['state'].fillna('No Disponible')

business.loc[:, 'attributes'] = business['attributes'].fillna('No Disponible')

business.loc[:, 'hours'] = business['hours'].fillna('No Disponible')

business.loc[:, 'categories'] = business['categories'].fillna("")

In [ ]:
business.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,No Disponible,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",No Disponible


In [ ]:
business.isna().sum()

,0
business_id,0
name,0
address,0
city,0
state,0
postal_code,0
latitude,0
longitude,0
stars,0
review_count,0


Empiezo a Crear la Tabla 'yelp_categories_business'

In [ ]:
categories = business[['business_id', 'categories']]

La columna 'categories' tiene un String con todas las categorias separadas por una coma.  Voy a crear una columna que sea una Lista de Python a partir de la columna 'categories' para luego aprovechar las bondades del metodo 'explode' y crear la tabla

In [ ]:
categories.loc[:, 'lista_categorias'] = categories['categories'].apply(lambda x: x.split(','))

categories = categories.drop(columns=['categories'])

categories.head()

,business_id,lista_categorias
0,Pns2l4eNsfO8kk83dixA6A,"[Doctors, Traditional Chinese Medicine, Natu..."
1,mpf3x-BjTdTEA3yCZrAYPw,"[Shipping Centers, Local Services, Notaries,..."
2,tUFrWirKiKi_TAnsVWINQQ,"[Department Stores, Shopping, Fashion, Home..."
3,MTSW4McQd7CbVtyjqoe9mw,"[Restaurants, Food, Bubble Tea, Coffee & Te..."
4,mWMc6_wTdE0EUBKIGXDVfA,"[Brewpubs, Breweries, Food]"


Desempaquetando la Lista de Categorias

In [ ]:
categories = categories.explode('lista_categorias')

categories.head()

,business_id,lista_categorias
0,Pns2l4eNsfO8kk83dixA6A,Doctors
0,Pns2l4eNsfO8kk83dixA6A,Traditional Chinese Medicine
0,Pns2l4eNsfO8kk83dixA6A,Naturopathic/Holistic
0,Pns2l4eNsfO8kk83dixA6A,Acupuncture
0,Pns2l4eNsfO8kk83dixA6A,Health & Medical


Salvando lo que va de Tabla 'yelp_categories_business` pero como todavia no esta creado el maestro de categorias lo dejo hasta aca

In [ ]:
categories.to_parquet('datasets/yelp/categories_business_yelp_todavia_nombres.parquet')

del categories

Ya guardamos `categories` toca seguir con `attributes`

In [ ]:
attributes = business[['business_id', 'attributes']]
attributes.head()

,business_id,attributes
0,Pns2l4eNsfO8kk83dixA6A,{'ByAppointmentOnly': 'True'}
1,mpf3x-BjTdTEA3yCZrAYPw,{'BusinessAcceptsCreditCards': 'True'}
2,tUFrWirKiKi_TAnsVWINQQ,"{'BikeParking': 'True', 'BusinessAcceptsCredit..."
3,MTSW4McQd7CbVtyjqoe9mw,"{'RestaurantsDelivery': 'False', 'OutdoorSeati..."
4,mWMc6_wTdE0EUBKIGXDVfA,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc..."


Elimino los registro `No Disponible` de esta tabla `attributes`

In [ ]:
attributes = attributes[attributes['attributes'] != 'No Disponible']

Funcion para crear Listas de String a partir Diccionarios

In [ ]:
def dict_to_list(diccionario:dict) -> list:
    """dict_to_list: Recibe un Diccionario y lo devuelve convertido en Lista

    Args:
        diccionario (dict): Diccionario de la columna attributes

    Returns:
        list: lista resultante de convertir la columna `attributes` a lista
    """
    return [f"{key}:{value}" for key, value in diccionario.items()]

In [ ]:
attributes['lista_attributes'] = attributes['attributes'].apply(dict_to_list)
attributes.head()

,business_id,attributes,lista_attributes
0,Pns2l4eNsfO8kk83dixA6A,{'ByAppointmentOnly': 'True'},[ByAppointmentOnly:True]
1,mpf3x-BjTdTEA3yCZrAYPw,{'BusinessAcceptsCreditCards': 'True'},[BusinessAcceptsCreditCards:True]
2,tUFrWirKiKi_TAnsVWINQQ,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","[BikeParking:True, BusinessAcceptsCreditCards:..."
3,MTSW4McQd7CbVtyjqoe9mw,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","[RestaurantsDelivery:False, OutdoorSeating:Fal..."
4,mWMc6_wTdE0EUBKIGXDVfA,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","[BusinessAcceptsCreditCards:True, WheelchairAc..."


Debo borrar columna `attributes`, luego hacer explode en columna `lista_attributes` y por último renombrar la columna

In [ ]:
attributes = attributes.drop(columns=['attributes'])

attributes = attributes.explode('lista_attributes')

attributes.rename(columns={'lista_attributes': 'attributes'}, inplace=True)

attributes.head()

,business_id,attributes
0,Pns2l4eNsfO8kk83dixA6A,ByAppointmentOnly:True
1,mpf3x-BjTdTEA3yCZrAYPw,BusinessAcceptsCreditCards:True
2,tUFrWirKiKi_TAnsVWINQQ,BikeParking:True
2,tUFrWirKiKi_TAnsVWINQQ,BusinessAcceptsCreditCards:True
2,tUFrWirKiKi_TAnsVWINQQ,RestaurantsPriceRange2:2


Guardo la table yelp.attributes_business

In [ ]:
attributes.to_parquet('datasets/yelp/yelp_attributes_business.parquet')

del attributes

Ahora debo terminar la Tabla `yelp_business`


Se deben borrar las Columnas de `attributes` y `categories` que ya las tenemos en tablas aparte

In [ ]:
business = business.drop(columns=['attributes', 'categories'])
business.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,No Disponible,93101,34.426679,-119.711197,5.0,7,0,No Disponible


Debo crear una funcion que retorne los Horarios de los distintos dias

In [ ]:
def retornar_horario(data:dict|str, dia:str) -> str:
  """retornar_horario: ayuda a crear las columnas monday, tuesday....

  Args:
      data (dict | str): El diccionario que hay en la columna `hours` o "No Disponible"
      dia (str): El nombre de la columna(dia) que se está creando

  Returns:
      str: Horario del dia específico que se está creando en la tabla
  """
  if data == 'No Disponible':
    return 'No Disponible'
  else:
    if dia in data.keys():
      return data[dia]
    else:
      return "No Disponible"

Creando las Columnas de los distintos dias

In [ ]:
business['monday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Monday'))

business['tuesday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Tuesday'))

business['wednesday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Wednesday'))

business['thursday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Thursday'))

business['friday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Friday'))

business['saturday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Saturday'))

business['sunday'] = business['hours'].apply(lambda x: retornar_horario(x, dia='Sunday'))

business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,hours,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,No Disponible,93101,34.426679,-119.711197,5.0,7,0,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible,No Disponible
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,No Disponible,63123,38.551126,-90.335695,3.0,15,1,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,No Disponible
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,No Disponible,85711,32.223236,-110.880452,3.5,22,0,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",No Disponible,No Disponible,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0


Ahora de se debe borrar la columna `hours` y se guarda la tabla `yelp_business`

In [ ]:
business = business.drop(columns=['hours'])

business.to_parquet('datasets/yelp/yelp_business.parquet')